## Experiment 11 - BERT Implementation
___

Using hugging face pretrained models.  
BERT stands for Bidirectional Encoder Representations from Transformers.

In [2]:
import torch
from transformers import BertConfig, BertModel, BertTokenizer

C:\Users\kannu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config = BertConfig()
model = BertModel(config)

config = model.config

In [4]:
sample = input("Enter a string: ")
print(sample)

Glw


In [5]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# Tokenize input text
tokens = tokenizer.encode(sample, add_special_tokens=True)

In [7]:
# Convert tokenized sequence to PyTorch tensors
tokens_tensor = torch.tensor(tokens).unsqueeze(0)

In [8]:
# Pass the input through BERT model to get the outputs
outputs = model(tokens_tensor)

In [9]:
# Get the output embeddings for the [CLS] token
# which can be used as a fixed-dimensional representation of the input sentence
cls_embedding = outputs.last_hidden_state[:, 0, :]

print(cls_embedding)

tensor([[-2.2247e-01,  1.0307e-01,  2.5152e-01, -1.2580e-01, -1.2775e-01,
         -5.5913e-02,  1.1254e-01,  1.9855e-01, -7.8039e-02, -1.1481e-01,
         -1.9383e-01,  1.4925e-01, -3.0677e-02,  5.8242e-01,  1.6761e-01,
          2.0463e-01, -3.6064e-01,  7.1997e-01,  2.4431e-01, -3.2773e-01,
         -1.0900e-01, -3.3103e-01,  1.8216e-02, -2.5717e-01, -6.3992e-02,
          3.2706e-02, -3.0477e-01, -1.5038e-01,  1.9739e-02,  6.6883e-02,
         -1.1199e-01,  2.6141e-01, -1.6487e-01,  1.5882e-01,  1.8117e-01,
         -5.7753e-02,  9.7379e-03, -2.1664e-01, -2.8773e-02,  3.6730e-01,
          1.9354e-01, -5.8425e-02,  3.0704e-01, -8.3526e-02, -2.5457e-02,
         -4.0698e-01, -2.1001e+00, -1.3526e-01, -1.7423e-01, -2.9494e-01,
          1.6362e-02, -4.2271e-02, -2.0071e-03,  3.0743e-02, -2.7362e-01,
          5.0478e-01, -2.2450e-01,  4.1450e-01,  3.2897e-01, -1.4867e-01,
         -4.7380e-02, -1.5123e-01, -1.3995e-01,  3.7243e-01, -9.1023e-02,
          1.6679e-01,  1.7505e-02,  4.

In [11]:
cls_embedding.size()

torch.Size([1, 768])

Here is an implementation where BERT tries to fill in the masked token.

This is also a task on which the BERT model was trained. It was done using a technique called Masked Language Modelling (MLM). In this, a small percentage of the tokens passed during training BERT, were randomly masked (hidden), and it was BERT's task to fill them in, based on the context and sentence.  

The B, standing for Bidirectional, means that the model looks for context in both directions.

Another task that BERT was trained on, was Next Sentence Prediction (NSP). In this task, BERT is trained to predict whether a given pair of sentences are consecutive sentences in a document, or not.

This also allowed the model to learn the relationship between different sentences in a document.

In [ ]:
from transformers import BertForMaskedLM
from torch.nn import functional as F

model = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict = True)
text = "Hey! He " + tokenizer.mask_token + " my cheeseburger."
input = tokenizer.encode_plus(text, return_tensors = "pt")
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
logits = model(**input)
logits = logits.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_5 = torch.topk(mask_word, 5, dim = 1)[1][0]
for token in top_5:
   word = tokenizer.decode([token])
   new_sentence = text.replace(tokenizer.mask_token, word)
   print(new_sentence)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Hey! He was my cheeseburger.
Hey! He ate my cheeseburger.
Hey! He is my cheeseburger.
Hey! He stole my cheeseburger.
Hey! He saved my cheeseburger.
